# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

In [15]:
!pip install zeno-client
!pip install --upgrade pip
!pip install --upgrade bottleneck
!pip install langdetect


In [16]:
!python --version

Python 3.10.13


In [17]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

# Initialize a client with our API key.
client = ZenoClient("zen_JxwDHAtiJnbO6Ai99obLcv_J8WDBPXorC2cEXrNeHvw")


# Step 2 - Create a project

In [18]:

project = client.create_project(
    name="Biased Movies Analysis_sauravy",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
        #Uncomment the following line after implementing the function
        ZenoMetric(name="avg length ratio", type="mean", columns=["avg_length_ratio"]),
    ]
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/fea4b94a-b0c7-47bf-b556-b6ea9d0043a2/Biased%20Movies%20Analysis_sauravy


# Step 3 - Create dataset

In [19]:
import pandas as pd
from langdetect import detect
from collections import Counter
import random

# Sample language detection function
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"

# Create a Pandas DataFrame for the biased dataset of movies
df = pd.DataFrame(
    {
        "id": range(1, 11),  # Using only 10 instances for demonstration
        "text": [
            "I love this movie!",
            "¡Odio esta película!",
            "Ce film est ok.",
            "The acting was great!",
            "The plot was confusing.",
            "I really enjoyed it!",
            "Worst movie ever!",
            "Not bad, but could be better.",
            "A masterpiece!",
            "I didn't like it.",
        ],
        "genre": ["action", "drama", "comedy", "action", "drama", "action", "drama", "comedy", "action", "drama"],
        "year": [random.choice([2020, 2021, 2022]) for _ in range(10)],  # Randomly select year
        "label": ["positive", "negative", "neutral", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Introduce biases in genre
biased_genre = ["action", "action", "drama", "drama", "comedy", "comedy", "comedy", "comedy", "drama", "drama"]
df["biased_genre"] = biased_genre

# Introduce biases in year
biased_year = [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022, 2022]
df["biased_year"] = biased_year

# Add language detection to create a new column
df["detected_language"] = df["text"].apply(detect_language)

# Add additional columns for analysis
df["input_length"] = df["text"].str.len()

#Uncomment the following line
# df["avg_length_ratio"] = df["input_length"]/(df["input_length"].sum()/len(df))
df["avg_length_ratio"] = df["text"].apply(lambda x: len(x.split(' '))/len(x))



In [20]:
df["avg_length_ratio"]

0    0.222222
1    0.150000
2    0.266667
3    0.190476
4    0.173913
5    0.200000
6    0.176471
7    0.206897
8    0.142857
9    0.235294
Name: avg_length_ratio, dtype: float64

# Step 4 - Upload the Base Dataset


In [21]:

project.upload_dataset(df, id_column="id", data_column="text", label_column="label")



/Users/sauravyadav/Documents/Repos/cmu-mlip-zeno-lab/env/lib/python3.10/site-packages/zeno_client/util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, id_column] = df[id_column].astype(str)
100%|██████████| 1/1 [00:00<00:00,  1.90it/s]

Successfully uploaded data


# Step 5 - Upload the AI System Outputs


In [22]:
# For demonstration, let's assume we have system predictions
df_system = pd.DataFrame(
    {
        "output": ["positive", "negative", "negative", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Create an id column to match the base dataset
df_system["id"] = df_system.index + 1

# Measure accuracy for each instance, which is averaged by the ZenoMetric above
df_system["correct"] = (df_system["output"] == df["label"]).astype(int)



# Step 6 - Upload the system outputs


In [23]:
project.upload_system(df_system, name="System A", id_column="id", output_column="output")

/Users/sauravyadav/Documents/Repos/cmu-mlip-zeno-lab/env/lib/python3.10/site-packages/zeno_client/util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, id_column] = df[id_column].astype(str)
100%|██████████| 1/1 [00:00<00:00,  1.94it/s]

Successfully uploaded system
